In [1]:
import numpy as np
import xarray as xr
import matplotlib as mpl
import matplotlib.pyplot as plt
from datetime import datetime
from IPython.display import Image
import scipy.io
import pandas as pd
from IPython.display import display, clear_output
import sys, os
import warnings
import h5py
#from fcmeans import FCM
warnings.filterwarnings('ignore')
from sklearn.mixture import GaussianMixture
from sklearn.mixture import BayesianGaussianMixture
from scipy import linalg
import itertools
#get_ipython().run_line_magic('matplotlib', 'inline')
from aidapy import load_data
import aidapy.aidaxr
from scipy.interpolate import griddata
from sklearn import mixture
from scipy.ndimage import gaussian_filter

In [45]:
#set time
start_time = datetime(2017, 10, 16, 13, 7, 0);
end_time   = datetime(2017, 10, 16, 13, 8, 0);

#choose mms probe
probes= 1

#various
vmax = 600

#dimension of interpolation grid
grid_dim=50

#h5py file name
filename = 'outto_tst.h5'

#max components for gmm
n_components_range= 15

#number of generated particles from vdf
n_part=40000

#choose between bic or aic
information_criterion='bic'

In [46]:
settings = {'prod': ['i_dist','sc_pos','sc_att'],
            'probes': probes, 'coords': 'gse', 'mode': 'high_res', 'frame':'gse'}

xr_mms = load_data(mission='mms', start_time=start_time, end_time=end_time, **settings)
xr_mms

68490it [00:33, 2048.92it/s]
19245it [00:02, 7121.50it/s] 


Creating new directory /home/pieterjan/heliopy/data/mms/1/mec/brst


21972it [00:26, 833.45it/s] 


<xarray.Dataset>
Dimensions:                   (time1: 0, mms1_dis_energy_brst: 32,
                               mms1_dis_theta_brst: 16, mms1_dis_phi_brst: 32,
                               time2: 0, mms1_mec_r_gse: 3, time3: 3,
                               mms1_mec_quat_eci_to_gse: 4)
Coordinates:
  * time1                     (time1) datetime64[ns] 
  * mms1_dis_energy_brst      (mms1_dis_energy_brst) float32 2.16 ... 2.83e+04
  * mms1_dis_theta_brst       (mms1_dis_theta_brst) float32 5.625 ... 174.4
  * mms1_dis_phi_brst         (mms1_dis_phi_brst) float32 1.75 13.0 ... 350.5
  * time2                     (time2) datetime64[ns] 
  * mms1_mec_r_gse            (mms1_mec_r_gse) <U1 'x' 'y' 'z'
  * time3                     (time3) datetime64[ns] 2017-10-16T13:07:00 ... ...
  * mms1_mec_quat_eci_to_gse  (mms1_mec_quat_eci_to_gse) <U3 'x' 'y' 'z' 'tot'
Data variables:
    i_dist1                   (time1, mms1_dis_energy_brst, mms1_dis_theta_brst, mms1_dis_phi_brst) float32 ...
    sc_pos1                   (time2, mms1_mec_r_gse) float64 
    sc_att1                   (time3, mms1_mec_quat_eci_to_gse) float64 -0.04...
Attributes:
    mission:        mms
    load_settings:  {'prod': ['i_dist', 'sc_pos', 'sc_att'], 'probes': 1, 'co...

In [50]:
xr_mms['sc_pos1']

<xarray.DataArray 'sc_pos1' (time2: 0, mms1_mec_r_gse: 3)>
array([], shape=(0, 3), dtype=float64)
Coordinates:
  * time2           (time2) datetime64[ns] 
  * mms1_mec_r_gse  (mms1_mec_r_gse) <U1 'x' 'y' 'z'
Attributes:
    Units:    {'mms1_mec_r_gse': 'km'}

In [37]:
print(np.array(xr_mms['sc_att3']))

[[-0.04069716 -0.19898368  0.95929615  0.1962145 ]
 [-0.04069776 -0.19898356  0.95929556  0.19621738]
 [-0.04069836 -0.19898344  0.95929497  0.19622027]]


In [42]:
print(np.array(xr_mms['time3']))

['2017-10-16T13:07:00.000000000' '2017-10-16T13:07:30.000000000'
 '2017-10-16T13:08:00.000000000']


1.009595072293838